<a href="https://www.kaggle.com/code/samuelkabati1/identifying-age-related-conditions?scriptVersionId=153695304" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd 
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier 
from keras import models
from keras import layers
from keras import regularizers
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
df=pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [3]:
test=pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
test.tail()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
plot_df = df.Class.value_counts()
plot_df

0    509
1    108
Name: Class, dtype: int64

In [5]:
#dropping ID column as it will not be used 
#also drop the predictor 

train_data=df.drop(columns=['Id','Class'])
train_data.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614


In [6]:
X=train_data
X

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,0.149555,3130.05946,123.763599,9.513984,13.020852,3.499305,0.077343,8.545512,2.804172,4157.68439,...,9.879296,0.173229,1.26092,0.067730,8.967128,217.148554,8095.932828,24.640462,69.191944,21.978000
613,0.435846,5462.03438,85.200147,46.551007,15.973224,5.979825,0.025882,12.622906,3.777550,5654.07556,...,10.910227,10.223150,1.24236,0.426699,35.896418,496.994214,3085.308063,29.648928,124.808872,0.145340
614,0.427300,2459.10720,130.138587,55.355778,10.005552,8.070549,0.025578,15.408390,1.229900,5888.87769,...,12.029366,0.173229,0.49706,0.067730,19.962092,128.896894,6474.652866,26.166072,119.559420,21.978000
615,0.363205,1263.53524,85.200147,23.685856,8.138688,7.981959,0.025578,7.524588,1.229900,4517.86560,...,8.026928,9.256996,0.78764,0.670527,24.594488,72.611063,1965.343176,25.116750,37.155112,0.184622


In [7]:
#The predictor
y=df['Class']


In [8]:
y_true=y.values


In [9]:
y_true

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,

# Building a naive model

 The naive model would be that every patient has a 17.5% probability to have one of the three age related conditions. We will need to compute the balanced log loss for the naive model and use it as the baseline balanced log loss. Any other models that we make should have a balanced log loss that is less than that of our naive model.

In [10]:
class_zero=509 #frequency of class zero
class_one=108 #frequency of class one 
num_rows=class_zero+class_one
prob_of_one=class_one/(class_one+class_zero)
prob_of_one_percentage=(class_one/(class_one+class_zero))*100
prob_of_one_percentage

17.504051863857377

In [11]:
prob_of_zero=class_zero/(class_one+class_zero)
prob_of_zero_percentage=(class_zero/(class_one+class_zero))*100
prob_of_zero_percentage

82.49594813614263

In [12]:
 #This means we create an array of length freq_class_0 + freq_class_1 where each element is prob_class_1 .
y_pred = np.array([prob_of_one] * num_rows)
y_pred

array([0.17504052, 0.17504052, 0.17504052, 0.17504052, 0.17504052,
       0.17504052, 0.17504052, 0.17504052, 0.17504052, 0.17504052,
       0.17504052, 0.17504052, 0.17504052, 0.17504052, 0.17504052,
       0.17504052, 0.17504052, 0.17504052, 0.17504052, 0.17504052,
       0.17504052, 0.17504052, 0.17504052, 0.17504052, 0.17504052,
       0.17504052, 0.17504052, 0.17504052, 0.17504052, 0.17504052,
       0.17504052, 0.17504052, 0.17504052, 0.17504052, 0.17504052,
       0.17504052, 0.17504052, 0.17504052, 0.17504052, 0.17504052,
       0.17504052, 0.17504052, 0.17504052, 0.17504052, 0.17504052,
       0.17504052, 0.17504052, 0.17504052, 0.17504052, 0.17504052,
       0.17504052, 0.17504052, 0.17504052, 0.17504052, 0.17504052,
       0.17504052, 0.17504052, 0.17504052, 0.17504052, 0.17504052,
       0.17504052, 0.17504052, 0.17504052, 0.17504052, 0.17504052,
       0.17504052, 0.17504052, 0.17504052, 0.17504052, 0.17504052,
       0.17504052, 0.17504052, 0.17504052, 0.17504052, 0.17504

In [13]:
#we are basically predicting class zero all the time 
y_pred_classes = np.array([0] * num_rows)
y_pred_classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
#normal log_loss
normal_log_loss=log_loss(y_true,y_pred)
normal_log_loss

0.46378926223013367

In [15]:
#balanced log loss formula
def b_log_loss(y_t, y_p):
    nc = np.bincount(y_t)
    sample_weight = 1 / nc[y_t]
    return log_loss(y_t, y_p, sample_weight=sample_weight, eps=1e-15)

In [16]:
custom_scorer = make_scorer(b_log_loss, greater_is_better=False,needs_proba=True)
#scorer for the neural net as it does not have predict_proba method 
custom_scorer_n= make_scorer(b_log_loss, greater_is_better=False)

In [17]:
#balanced log loss 
balanced_log_loss= b_log_loss(y_true, y_pred)
balanced_log_loss

0.9675794020680168

# Preprocessing pipelines 

Only one column `EJ` is categorical.

In [18]:
train_data['EJ'].isna().sum()

0

In [19]:
train_data['EJ'].unique()

array(['B', 'A'], dtype=object)

In [20]:
#numerical pipeline
imputer=SimpleImputer(strategy='median')
log_transformer=FunctionTransformer(np.log1p, inverse_func=np.expm1)
scaler=StandardScaler()

numerical_pipeline=Pipeline([
    ('impute',imputer),
    ('log transform',log_transformer),
    ('scale', scaler)
])

In [21]:
#categorical pipeline
cat_encoder=OneHotEncoder()
cat_pipeline=Pipeline([
    ('encoder',cat_encoder)
])

In [22]:
X_preprocessing=ColumnTransformer([
    ('numerical', numerical_pipeline,make_column_selector(dtype_include=np.number)),
    ('categorical',cat_pipeline, make_column_selector(dtype_include=object) )
])

# Modelling

The data is not much so we will use k-fold  cross validation. We will make use of the pipeline to avoid data leakage.

In [23]:
#cross-validation strategy
cv=KFold(n_splits=10, shuffle=True, random_state=42)

In [24]:
#logistic regression
logreg=LogisticRegression(fit_intercept=False, C=1e12, solver='liblinear', class_weight='balanced')
#logistic regression training pipeline 
logreg_pipeline= Pipeline([
    ('preprocess', X_preprocessing),
    ('classifier',logreg)
])
scores = cross_val_score(logreg_pipeline, X, y, cv=cv, scoring=custom_scorer)
# Calculate the average custom score
average_custom_score = -scores.mean()
average_custom_score

0.8401360735921515

In [25]:
dt= DecisionTreeClassifier(random_state=42)
dt_pipeline=Pipeline([
    ('preprocessing', X_preprocessing),
    ('model',dt)
    
] 
)
scores = cross_val_score(dt_pipeline, X, y, cv=cv, scoring=custom_scorer)
# Calculate the average custom score
average_custom_score = -scores.mean()
average_custom_score

8.611315859177273

In [26]:
knn=KNeighborsClassifier() 
knn_pipeline=Pipeline([
    ('preprocessing', X_preprocessing),
    ('model',knn)
])
scores = cross_val_score(knn_pipeline, X, y, cv=cv, scoring=custom_scorer)
# Calculate the average custom score
average_custom_score = -scores.mean()
average_custom_score

2.502229221630289

In [27]:
#remember we cannot use rf and xgboost but we can uae them to compare with the dinal model we come up with 
rf=RandomForestClassifier(random_state=42)
rf_pipeline=Pipeline([
    ('preprocessing',X_preprocessing),
    ('model',rf)
])
scores = cross_val_score(rf_pipeline, X, y, cv=cv, scoring=custom_scorer)
# Calculate the average custom score
average_custom_score = -scores.mean()
average_custom_score

0.4449667909765969

In [28]:
#xgboost#check log loss, hence balanced log loss 
xg_boost=GradientBoostingClassifier(random_state=42)
xg_pipeline=Pipeline([
    ('preprocessing',X_preprocessing),
    ('model',xg_boost)
])
scores = cross_val_score(xg_pipeline, X, y, cv=cv, scoring=custom_scorer)
# Calculate the average custom score
average_custom_score = -scores.mean()
average_custom_score

0.46790016116872435

In [29]:
et=ExtraTreesClassifier(random_state=42,n_estimators=12)
et_pipeline=Pipeline([
    ('preprocessing',X_preprocessing),
    ('model',et)
])
scores = cross_val_score(et_pipeline, X, y, cv=cv, scoring=custom_scorer)
# Calculate the average custom score
average_custom_score = -scores.mean()
average_custom_score

0.7812817199704217

In [30]:
ab=AdaBoostClassifier()
ab_pipeline=Pipeline([
    ('preprocessing',X_preprocessing),
    ('model',ab),
  
])
scores = cross_val_score(ab_pipeline, X, y, cv=cv, scoring=custom_scorer)
# Calculate the average custom score
average_custom_score = -scores.mean()
average_custom_score

0.6379101745050535

In [31]:
hg=HistGradientBoostingClassifier(random_state=42)
hg_pipeline=Pipeline([
    ('preprocessing',X_preprocessing),
    ('model',hg)
])
scores = cross_val_score(hg_pipeline, X, y, cv=cv, scoring=custom_scorer)
# Calculate the average custom score
average_custom_score = -scores.mean()
average_custom_score

0.45040581561254667

In [32]:
X_processed=X_preprocessing.fit_transform(X)
n_features=X_processed.shape[1]
n_features

57

Finetuning three models with the best balanced log loss

- RandomForest
- GradientBoosting 
- HistGradientBoosting

In [33]:
#tuning the gradient boosting classifier
xg_estimator=GradientBoostingClassifier(random_state=42)
xg_boost_grid={
    'n_estimators':[50,60,70,80,90,100,120,130,140,150,150,170,180,190,200],
    'subsample':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
    'learning_rate':[0.125,0.25,0.5,0.75,1.0],
    'min_samples_split':[2,4,6,8,10,12],
    'min_samples_leaf':[2,4,6,8,10,12],
    'min_weight_fraction_leaf':[0.1,0.2,0.3,0.4,0.5],
    'max_depth':[1,2,3,4,5,6,7,8,9,10,11,12]
}
xg_random=RandomizedSearchCV(estimator=xg_estimator,
                             param_distributions=xg_boost_grid,
                             n_iter=75,
                             cv=cv,
                             random_state=42,
                             n_jobs=-1)
#xg_random.fit(X_processed,y)

In [34]:
#print(f"Best Parameter Combination Found:{xg_random.best_params_}")

The best combination from Random Search was:
- subsample: 0.9
- n_estimators: 140
- min_weight_fraction_leaf: 0.1
- min_samples_split: 8
- max_depth: 5
- learning_rate: 0.125

In [35]:
xg_tuned=GradientBoostingClassifier(random_state=42, subsample=0.9, n_estimators=140, min_weight_fraction_leaf=0.1, min_samples_split=8, min_samples_leaf=4, max_depth=5, learning_rate=0.125)
xg_tuned_pipeline=Pipeline([
    ('preprocessing',X_preprocessing),
    ('model',xg_tuned),
  
])
scores = cross_val_score(xg_tuned_pipeline, X, y, cv=cv, scoring=custom_scorer)
# Calculate the average custom score
average_custom_score = -scores.mean()
average_custom_score

0.3774958214802123

The tuned gradient boosting model performs much better than the baseline gradient boosting model

In [36]:
#tuning the histogram-based gradient boosting classifier
hg_estimator=HistGradientBoostingClassifier(random_state=42, class_weight='balanced')
hg_estimator_grid={
    'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
}
hg_grid=GridSearchCV(
    estimator=hg_estimator,
    param_grid=hg_estimator_grid,
    cv=cv,
    n_jobs=-1,
)
#hg_grid.fit(X_processed,y)

In [37]:

#print(f"Best Parameter Combination Found:{hg_grid.best_params_}")

The best parameters were:
- learning_rate: 0.2

In [38]:
hg_tuned=HistGradientBoostingClassifier(random_state=42, class_weight='balanced',learning_rate=0.2)
hg_tuned_pipeline=Pipeline([
    ('preprocess', X_preprocessing),
    ('model', hg_tuned )
])
scores = cross_val_score(hg_tuned_pipeline, X, y, cv=cv, scoring=custom_scorer)
# Calculate the average custom score
average_custom_score = -scores.mean()
average_custom_score

0.5325303569946698

The tuned histogram-based gradient boosting model seems to be doing worse than the baseline. We will stick to the baseline but have `class_weight` as balanced.

In [39]:
hg_balanced=HistGradientBoostingClassifier(random_state=42, class_weight='balanced')
hg_balanced_pipeline=Pipeline([
    ('preprocess', X_preprocessing),
    ('model', hg_balanced)
])
scores = cross_val_score(hg_balanced_pipeline, X, y, cv=cv, scoring=custom_scorer)
# Calculate the average custom score
average_custom_score = -scores.mean()
average_custom_score

0.3635252849109752

In [40]:
#tuning the random forest classifier
rf_estimator=RandomForestClassifier(random_state=42,n_jobs=-1)
rf_grid={
'criterion':['entropy','log_loss','gini'],
'class_weight':['balanced','balanced_subsample'],
'min_samples_leaf': range(1,50),
}
rf_random=RandomizedSearchCV(
    estimator=rf_estimator,
    param_distributions=rf_grid,
    n_jobs=-1,
    cv=cv
    )
#rf_random.fit(X_processed,y)

In [41]:
#print(f"Best Parameter Combination Found:{rf_random.best_params_}")

The best parameters for the random forest were:
- min_samples_leaf: 11
- criterion: log_loss
- class_weight: balanced subsample

In [42]:
rf_tuned=RandomForestClassifier(random_state=42,n_jobs=-1,min_samples_leaf=11, criterion='log_loss', class_weight='balanced_subsample')
rf_tuned_pipeline=Pipeline([
    ('preprocess', X_preprocessing),
    ('model', rf_tuned )
])
scores = cross_val_score(rf_tuned_pipeline, X, y, cv=cv, scoring=custom_scorer)
# Calculate the average custom score
average_custom_score = -scores.mean()
average_custom_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


0.38953259403735196

## Implementing SMOTE

In [43]:
#smote
from imblearn.over_sampling import SMOTE
sm= SMOTE(random_state=42)
X_oversampled, y_oversampled =sm.fit_resample(X_processed, y)
y_oversampled.value_counts()

1    509
0    509
Name: Class, dtype: int64

We are going to use Histogram Based Gradient Boosting to come up with the final model as it is the best performer so far. We are also going to evaluate the model's precision as it is very important in the medical field. A low precison score might indicate that our model is not good enough for practical use.

In [44]:
final_scorings = {'model_precision':'precision', 'balanced_log_loss' :custom_scorer }

In [45]:
#the histogram based gradient boosting with smote
hg_smote=HistGradientBoostingClassifier(random_state=42, class_weight='balanced')
hg_smote_pipeline=Pipeline([
    ('model', hg_smote)
])
scores = cross_validate(hg_smote_pipeline, X_oversampled, y_oversampled, cv=cv, scoring=final_scorings)
# Calculate the average custom score

print(f"Precision = {(scores['test_model_precision'].mean())}")
print('------------------')
print(f"Balanced Log Loss = {-(scores['test_balanced_log_loss'].mean())}")
print('------------------')

Precision = 0.9615364697368088
------------------
Balanced Log Loss = 0.08378296953502504
------------------


Precision is at 96% and balanced log loss at 0.08

In [46]:

final_model=hg_smote_pipeline.fit(X_oversampled, y_oversampled)

# Predicting test data 

In [47]:

test_data=test.drop(columns=['Id'])
test_data.head()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
X_test_processed=X_preprocessing.transform(test_data)

In [49]:
pred = final_model.predict_proba(X_test_processed)

In [50]:

submission=test[['Id']]
submission['class_0']=pred[:,0]
submission['class_1']=pred[:,1]
submission

/tmp/ipykernel_21/1709050890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['class_0']=pred[:,0]
/tmp/ipykernel_21/1709050890.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['class_1']=pred[:,1]


,Id,class_0,class_1
0,00eed32682bb,0.979315,0.020685
1,010ebe33f668,0.979315,0.020685
2,02fa521e1838,0.979315,0.020685
3,040e15f562a2,0.979315,0.020685
4,046e85c7cc7f,0.979315,0.020685


In [51]:
submission.to_csv("/kaggle/working/submission.csv",index=False)